Importujemy potrzebne biblioteki

In [11]:
#Zastosowane w piewszym i drugim modelu
from numpy import *


#Zastosowane w drugim modelu
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Wczytujemy dane z plików "horseColicTraining" oraz "horseColicTest" i zapisujemy je w zmiennych trainingSet, trainingLabels oraz testSet, testLabels.

In [12]:
def load_data(file_name):
    data_set = []
    data_labels = []
    with open(file_name) as fr:
        for line in fr.readlines():
            curr_line = line.strip().split('\t') # TAB
            line_arr = [float(curr_line[i]) for i in range(21)]  # 21 features
            data_set.append(line_arr)
            data_labels.append(float(curr_line[21]))  # label in the 22nd column
    return array(data_set), array(data_labels)

# Load datasets
trainingSet, trainingLabels = load_data('horseColicTraining.txt')
testSet, testLabels = load_data('horseColicTest.txt')


Zaimplementowanie modelu regresji logistycznej na przykładzie podanym na https://github.com/pbharrin/machinelearninginaction

In [13]:
def sigmoid(inX): # funkcja logistyczna
    return 1.0 / (1 + exp(-inX))

def classifyVector(inX, weights):
    prob = sigmoid(sum(inX * weights))
    return 1.0 if prob > 0.5 else 0.0

def LogRegress(dataMatrix, classLabels, numIter=150):
    m, n = shape(dataMatrix)
    weights = ones(n)   # Initialize weights to all ones
    for j in range(numIter):
        dataIndex = list(range(m))
        for i in range(m):
            alpha = 4 / (1.0 + j + i) + 0.0001  # Alpha decreases with each iteration
            randIndex = int(random.uniform(0, len(dataIndex)))  # Randomly select index
            h = sigmoid(sum(dataMatrix[randIndex] * weights))
            error = classLabels[randIndex] - h
            weights = weights + alpha * error * dataMatrix[randIndex]
            del(dataIndex[randIndex])  # Remove the selected index
    return weights



Uczenie modelu i klasyfikacja predykcji oraz wyświetlenie spredyktowanych przez nasz model wartości w porównaniu do rzeczywistych wartości, a także jego dokładności.

In [14]:


trainWeights = LogRegress(array(trainingSet), trainingLabels, 1000)  # Train the model
    
errorCount = 0
numTestVec = 0.0
predictions = []  
actuals = [] 
    
for i in range(len(testSet)):
    numTestVec += 1.0
    lineArr = testSet[i]
    
    # Classify and store the prediction
    predicted = classifyVector(lineArr, trainWeights)
    predictions.append(predicted)
    actual = int(testLabels[i])
    actuals.append(actual)
    
    # Check if the prediction is correct
    if int(predicted) != actual:
        errorCount += 1

print("\nPredyktowane i rzeczywiste wartości:")
for predicted, actual in zip(predictions, actuals):
    print(f"Predyktowana: {predicted}, Rzeczywista: {actual}")
    
accuracyRate = 1 - (errorCount / numTestVec)
print(f"\nDokładność modelu: {accuracyRate * 100:.2f}%")   

C:\Users\harko\AppData\Local\Temp\ipykernel_22868\165168765.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + exp(-inX))



Predyktowane i rzeczywiste wartości:
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 0.0, Rzeczywista: 0
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 1.0, Rzeczywista: 1
Predyktowana: 0.0, Rzeczywista: 1
Predyktowa

Uczenie modelu LogisticRegression z biblioteki "scikit-learn". Wyświetlanie predyktowanych i rzeczywistych wartości oraz dokładności modelu.

In [15]:


# Train the logistic regression model
model = LogisticRegression(max_iter=1000)  # Use scikit-learn's Logistic Regression
model.fit(trainingSet, trainingLabels)  # Train the model

# Predict labels on the test set
test_predictions = model.predict(testSet)



# Show predicted and actual labels
print("\nPredyktowane i rzeczywiste wartości:")
for predicted, actual in zip(test_predictions, testLabels):
    print(f"Predyktowana: {predicted}, Rzeczywista: {actual}")

# Calculate accuracy
accuracy = accuracy_score(testLabels, test_predictions)
print(f"\nDokładność modelu scikit-learn: {accuracy * 100:.2f}%")




Predyktowane i rzeczywiste wartości:
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 0.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 0.0, Rzeczywista: 0.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 0.0, Rzeczywista: 0.0
Predyktowana: 0.0, Rzeczywista: 0.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 0.0, Rzeczywista: 0.0
Predyktowana: 0.0, Rzeczywista: 0.0
Predyktowana: 1.0, Rzeczywista: 0.0
Predyktowana: 0.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 0.0, Rzeczywista: 0.0
Predyktowana: 0.0, Rzeczywista: 0.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczywista: 1.0
Predyktowana: 1.0, Rzeczyw